# Tensorflow下mnist数据集的手写识别

# 1、导入包

In [7]:
import tensorflow as tf
import numpy as np

# 2、读入数据

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("d:\Mnist_data\MNIST_data", one_hot = True)

Extracting d:\Mnist_data\MNIST_data/train-images-idx3-ubyte.gz
Extracting d:\Mnist_data\MNIST_data/train-labels-idx1-ubyte.gz
Extracting d:\Mnist_data\MNIST_data/t10k-images-idx3-ubyte.gz
Extracting d:\Mnist_data\MNIST_data/t10k-labels-idx1-ubyte.gz


# 3、构建网络模型

#### 3.1常量定义

In [28]:
# ！！！避免magic number，提高程序可维护性

#图片高度
x_height = 28
#图片宽度
x_width = 28
#输入图片大小为28*28
n_inputs = 28 * 28
#输出类别0~9共10类
n_outputs = 10
#黑白图片，只有一个通道
in_channels = 1
#第一卷积层设置32个过滤器
n_filter_layer1 = 32
#第二个卷积层设置64个过滤器
n_filter_layer2 = 64
#第一全连接层的输入个数
n_inputs_full1 = 7*7*64
#第二全连接层的输入个数
n_inputs_full2 = 1024
#过滤器高度
filter_height = 5
#过滤器宽度
filter_width = 5

#### 3.2函数定义

In [15]:
#初始化权值
def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev = 0.1) 
    return tf.Variable(init)

#初始化偏置值
def bias_variable(shape):
    init = tf.constant(0.1, shape = shape)
    return tf.Variable(init)

#卷积计算
def conv2d(x, W):
    # x: input tensor of shape [batch, in_height, in_width, in_channels]
    # Filter / Kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    # strides[0]=strides[3]=1;strides[1]代表x方向的步长；strides[2]带表y方向的步长
    # padding：A 'string' from: 'SAME', 'VALID'
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')
    
#池化计算
def max_pool_2x2(x):
    #ksize [1, x, y, 1]
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

#### 3.3模型定义

In [16]:
#定义placeholder
x = tf.placeholder(tf.float32, [None, n_inputs]) # input
y = tf.placeholder(tf.float32, [None, n_outputs]) # lables
keep_prob = tf.placeholder(tf.float32)   #  probability for dropout

#修改输入格式
x_image = tf.reshape(tensor = x, shape = [-1, x_height, x_width, in_channels])

#第一卷积层权值和偏置值
W_conv1 = weight_variable(shape = [filter_height, filter_width, in_channels, n_filter_layer1])
b_conv1 = bias_variable(shape = [n_filter_layer1])
#第一层卷积
sigma_conv1 = conv2d(x_image, W_conv1) + b_conv1
#用relu进行非线性变换
relu_conv1 = tf.nn.relu(sigma_conv1)
#第一层池化
pool_conv1 = max_pool_2x2(relu_conv1)

#第二卷积层权值和偏置值
W_conv2 = weight_variable(shape = [filter_height, filter_width, n_filter_layer1, n_filter_layer2])
b_conv2 = bias_variable(shape = [n_filter_layer2])
#对第二层进行卷积
sigma_conv2 = conv2d(pool_conv1, W_conv2) + b_conv2
#用relu进行非线性变换
relu_conv2 = tf.nn.relu(sigma_conv2)
#对第二层进行池化
pool_conv2 = max_pool_2x2(relu_conv2)

#把池化层输出扁平化为1维（把数据从卷积层格式转换到全连接层输入格式）
pool_flat_conv2 = tf.reshape(tensor = pool_conv2, shape = [-1, n_inputs_full1])

#第一全连接层权值和偏置值
W_full1 = weight_variable(shape = [n_inputs_full1, n_inputs_full2])
b_full1 = bias_variable([n_inputs_full2])
#加法单元输出
sigma_full1 = tf.matmul(pool_flat_conv2, W_full1) + b_full1
#非线性单元输出
relu_full1 = tf.nn.relu(sigma_full1)
#dropout
drop_full1 = tf.nn.dropout(relu_full1, keep_prob)

#第二全连接层权值和偏置值
W_full2 = weight_variable(shape = [n_inputs_full2, n_outputs])
b_full2 = bias_variable([n_outputs])
#加法单元输出
sigma_full2 = tf.matmul(drop_full1, W_full2) + b_full2
#非线性单元输出(神经元网络输出)
prediction = tf.nn.softmax(sigma_full2)

#### 3.4损失函数

In [17]:
#交叉熵损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction))

#### 3.5最优化算法

In [29]:
#Adam优化算法
train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#### 3.6模型评价指标

In [20]:
#正确结果
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
#求准确率
accuracy = tf.reduce_mean(tf.cast(x = correct_prediction, dtype = tf.float32))

# 4、建立会话

#### 4.1常量定义

In [22]:
n_epochs = 21
batch_size = 100
n_batches = int(np.ceil(mnist.train.num_examples/batch_size))

#### 4.2建立会话

In [24]:
with tf.Session() as sess:
    #初始化所有变量
    sess.run(tf.global_variables_initializer())
    #迭代n_epochs次
    for epoch in range(n_epochs):
        #mini_batch=100
        for batch in range(n_batches)：
            #取一个batch的数据
            x_batch, y_batch = mnist.train.next_batch(batch_size = batch_size)
            #在训练集上训练模型
            sess.run(train, feed_dict = {x:x_batch, y:y_batch, keep_prob:.7})
        #在测试集上评价模型
        acc = sess.run(accuracy, feed_dict = {x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Epoch" + str(epoch) + ", Testing Accuracy:" + str(acc)) 

Epoch0; Testing Accuracy:0.8604
Epoch1; Testing Accuracy:0.8771
Epoch2; Testing Accuracy:0.8825
Epoch3; Testing Accuracy:0.8846
Epoch4; Testing Accuracy:0.8864
Epoch5; Testing Accuracy:0.8883
Epoch6; Testing Accuracy:0.8876
Epoch7; Testing Accuracy:0.8882
Epoch8; Testing Accuracy:0.986
Epoch9; Testing Accuracy:0.9871
Epoch10; Testing Accuracy:0.9888
Epoch11; Testing Accuracy:0.9889
Epoch12; Testing Accuracy:0.9902
Epoch13; Testing Accuracy:0.989
Epoch14; Testing Accuracy:0.989
Epoch15; Testing Accuracy:0.9906
Epoch16; Testing Accuracy:0.9907
Epoch17; Testing Accuracy:0.9915
Epoch18; Testing Accuracy:0.9915
Epoch19; Testing Accuracy:0.9907
Epoch20; Testing Accuracy:0.9921
